<a href="https://colab.research.google.com/github/inforeqd512/QLearning/blob/main/Tribe_Q_learning_v8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [300]:
import numpy as np
import copy #for deepcopy()
# importing functools for reduce()
import functools

import unittest
!python -V

Python 3.7.11


# Business Rules

In [301]:
class Business_rules:
  def __init__(self):
    self.target_dr_per_lead = 4
    self.target_max_reassign = 4
    return

# Chapter

In [302]:
class Allocated_leads:
  def __init__(self):
    self.allocated_leaders_gradewise = {"3":0, "4.1":0, "4.2":0}
    return

  def total_leaders_leading(self):
    total = functools.reduce(lambda x, value:x + value, self.allocated_leaders_gradewise.values(), 0)
    return total 

  def allocate_leaders(self, allocated_leaders_gradewise = {"3":0, "4.1":0, "4.2":0}):
    for grade in allocated_leaders_gradewise:
      self.allocated_leaders_gradewise[grade] += allocated_leaders_gradewise[grade]
    return

In [303]:
class Successor:
  def __init__(self, 
               people_remaining_gradewise = {"3":0, "4.1":2, "4.2":0, "4.3":0, "4.4":0, "5.1":0, "5.2":0, "MS":0},
               leader_successor_grades = {"3":"4.1", "4.1":"4.2", "4.2":"4.3"}
               ):
    self.people_remaining_gradewise = people_remaining_gradewise
    self.leader_successor_grades = leader_successor_grades
    self.engineer_successor_grades = {"4.3":"4.4", "4.4":"5.1", "5.1":"5.2"}
    return

  def successor_ratio_for_grade(self, kv_tuple):
    k = kv_tuple[0]
    v = kv_tuple[1]
    total_successors = self.people_remaining_gradewise[v]
    total_leaders = self.people_remaining_gradewise[k]
    ratio = 0
    if total_leaders <= 0:
      total_leaders = 1
    ratio = total_successors / total_leaders
    return ratio

  def dict_successor_per_lead(self):
    d = dict(map(lambda kv: (kv[0], self.successor_ratio_for_grade(kv)) , self.leader_successor_grades.items()))
    return d  


  def total_successors_for_lead_grades(self):
    d = self.dict_successor_per_lead()
    total = functools.reduce(lambda x, value: x + value, d.values(), 0)
    return total


  def dict_successor_per_engineer(self):
    d = dict(map(lambda kv: (kv[0], self.successor_ratio_for_grade(kv)) , self.engineer_successor_grades.items()))
    return d  


  def total_successors_for_engineer_grades(self):
    d = self.dict_successor_per_engineer()
    total = functools.reduce(lambda x, value: x + value, d.values(), 0)
    return total


  def diff_successor_ratio_for_leader_engineer_grades(self):
    l = self.total_successors_for_lead_grades()
    e = self.total_successors_for_engineer_grades()
    diff = l - e
    return diff

  def is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade(self):
    promotable_to_higher_grade = self.total_successors_for_lead_grades()
    promotable_to_lower_grade = self.total_successors_for_engineer_grades()
    diff = promotable_to_higher_grade - promotable_to_lower_grade
    rule = diff > 0
    return rule  

  def is_num_immediate_promotable_to_leader_grades_gt_zero(self):
    rule = self.total_successors_for_lead_grades() > 0
    return rule  

In [304]:
class DR_per_lead:
  def __init__(self, allocated_leads, business_rules, people_remaining_gradewise = {"3":0, "4.1":2, "4.2":0, "4.3":0, "4.4":0, "5.1":0, "5.2":0, "MS":0}):
    self.people_remaining_gradewise = people_remaining_gradewise
    self.allocated_leads = allocated_leads
    return

  def total_people_remaining(self):
    total = functools.reduce(lambda x, value:x + value, self.people_remaining_gradewise.values(), 0)
    return total

  def dr_per_lead(self):
    total = self.allocated_leads.total_leaders_leading()
    ratio = 999 #some infinitely big number
    if total > 0:
      ratio = self.total_people_remaining() / total
    return ratio

  def is_overall_dr_per_lead_gt_target(self):
    rule = self.dr_per_lead() > self.business_rules.target_dr_per_lead
    return rule

In [305]:
class DR_per_lead_for_this_leader_grade_wrt_higher_grades:
  def __init__(self, allocated_leads, business_rules, ranked_grades=["3","4.1","4.2"]):
    self.allocated_leads = allocated_leads
    self.ranked_grades = ranked_grades
    self.business_rules = business_rules
    return

  def dr_per_lead_for_this_grade_wrt_higher_grades(self, grade):
    total_leading_in_grade = self.allocated_leads.allocated_leaders_gradewise[grade]
    total_leading_in_higher_grades = 0
    for local_grade in self.ranked_grades:
      if local_grade == grade:
        break
      total_leading_in_higher_grades += self.allocated_leads.allocated_leaders_gradewise[local_grade]

    dr_per_lead_higher_grade = 0
    if total_leading_in_higher_grades > 0:
      dr_per_lead_higher_grade = total_leading_in_grade / total_leading_in_higher_grades

    return dr_per_lead_higher_grade

  def dict_dr_per_lead_for_this_grade_wrt_higher_grade(self):
    dict_dr = {}
    for grade in self.ranked_grades:
      dr_per_lead_higher_grade = self.dr_per_lead_for_this_grade_wrt_higher_grades(grade)
      dict_dr[grade] = dr_per_lead_higher_grade
    return dict_dr

  def max_dr_per_lead_in_leader_grades(self):
    vals = self.dict_dr_per_lead_for_this_grade_wrt_higher_grade().values()
    max_val = max(vals)
    return max_val

  def is_max_dr_per_lead_in_leader_grades_gt_target(self):
    rule = self.max_dr_per_lead_in_leader_grades() > self.business_rules.target_dr_per_lead
    return rule

In [306]:
class Chapter:
  def __init__(self, people_gradewise = {"3":0, "4.1":2, "4.2":0, "4.3":0, "4.4":0, "5.1":0, "5.2":0, "MS":0}):
    self.people_gradewise = people_gradewise
    self.allocated_leads = Allocated_leads()
    self.business_rules = Business_rules()
    self.people_remaining_gradewise = copy.copy(self.people_gradewise)
    self.leader_successor_grades = {"3":"4.1", "4.1":"4.2", "4.2":"4.3"}
    self.ranked_leader_grades = self.leader_successor_grades.keys()
    self.successor = Successor(people_remaining_gradewise = self.people_remaining_gradewise, 
                   leader_successor_grades = self.leader_successor_grades)
    self.dr_per_lead = DR_per_lead(people_remaining_gradewise = self.people_remaining_gradewise,
                                   business_rules = self.business_rules,
                                   allocated_leads = self.allocated_leads)
    self.dr_per_lead_for_this_leader_grade_wrt_higher_grades = DR_per_lead_for_this_leader_grade_wrt_higher_grades(allocated_leads = self.allocated_leads,
                                                                                                                   business_rules = self.business_rules)
    return
  
  def allocate_leaders(self, allocated_leaders_gradewise = {"3":0, "4.1":0, "4.2":0}):
    self.allocated_leads.allocate_leaders(allocated_leaders_gradewise=allocated_leaders_gradewise)
    for grade in allocated_leaders_gradewise:
      self.people_remaining_gradewise[grade] -= allocated_leaders_gradewise[grade]
    self.update_chapter_state()

  def update_chapter_state(self):
    self.successor = Successor(people_remaining_gradewise = self.people_remaining_gradewise, 
                   leader_successor_grades = self.leader_successor_grades)
    self.dr_per_lead = DR_per_lead(people_remaining_gradewise = self.people_remaining_gradewise,
                                  business_rules = self.business_rules,
                                  allocated_leads = self.allocated_leads)
    self.dr_per_lead_for_this_leader_grade_wrt_higher_grades = DR_per_lead_for_this_leader_grade_wrt_higher_grades(allocated_leads = self.allocated_leads,
                                                                                                                business_rules = self.business_rules)

  def total_people(self):
    print(self.people_gradewise)
    total = functools.reduce(lambda x, value:x + value, self.people_gradewise.values(), 0)
    return total

  def reassign_leaders(self):
    d = self.dict_leaders_that_can_be_reassigned()
    self.allocate_leaders(allocated_leaders_gradewise = d)
    return

  def dict_leaders_that_can_be_reassigned(self):
    d = dict(map(lambda k: (k, self.people_remaining_gradewise[k]) , self.ranked_leader_grades))
    return d

  def total_leaders_that_can_be_reassigned(self):
    total = functools.reduce(lambda x, k:x + self.people_remaining_gradewise[k], self.ranked_leader_grades, 0)
    return total

  def is_overall_dr_per_lead_gt_target(self):
    rule = self.dr_per_lead.dr_per_lead() > self.business_rules.target_dr_per_lead
    return rule

  def is_num_reassignable_leaders_gt_zero(self):
    rule = self.total_leaders_that_can_be_reassigned() > 0
    return rule

  def produce_state(self):
    state = State()
    state.is_overall_dr_per_lead_gt_target = self.is_overall_dr_per_lead_gt_target()
    state.is_num_reassignable_leaders_gt_zero = self.is_num_reassignable_leaders_gt_zero()
    state.is_num_immediate_promotable_to_leader_grades_gt_zero = self.successor.is_num_immediate_promotable_to_leader_grades_gt_zero()
    state.is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade = self.successor.is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade()
    state.is_max_dr_per_lead_in_leader_grades_gt_target = self.dr_per_lead_for_this_leader_grade_wrt_higher_grades.is_max_dr_per_lead_in_leader_grades_gt_target()
    return state


# State

In [307]:
class State:

  def __init__(self, is_overall_dr_per_lead_gt_target = False, 
                is_num_reassignable_leaders_gt_zero = False,
                is_num_immediate_promotable_to_leader_grades_gt_zero = False,
                is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade = False,
                is_max_dr_per_lead_in_leader_grades_gt_target = False):
    self.is_overall_dr_per_lead_gt_target = is_overall_dr_per_lead_gt_target
    self.is_num_reassignable_leaders_gt_zero = is_num_reassignable_leaders_gt_zero
    self.is_num_immediate_promotable_to_leader_grades_gt_zero = is_num_immediate_promotable_to_leader_grades_gt_zero 
    self.is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade = is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade
    self.is_max_dr_per_lead_in_leader_grades_gt_target = is_max_dr_per_lead_in_leader_grades_gt_target

  def state_copy(self):
    new_state = copy.deepcopy(self)
    return new_state

  def state_tuple(self):
    return ((self.is_overall_dr_per_lead_gt_target, 
                        self.is_num_reassignable_leaders_gt_zero, 
                        self.is_num_immediate_promotable_to_leader_grades_gt_zero, 
                        self.is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade,
                        self.is_max_dr_per_lead_in_leader_grades_gt_target))

  def state_hash(self):
    hash_value = hash((self.is_overall_dr_per_lead_gt_target, 
                        self.is_num_reassignable_leaders_gt_zero, 
                        self.is_num_immediate_promotable_to_leader_grades_gt_zero, 
                        self.is_num_people_immediate_promotable_to_higher_grade_lt_those_to_lower_grade,
                        self.is_max_dr_per_lead_in_leader_grades_gt_target))
    return hash_value



# Action

In [308]:
class Action:

  def __init__(self, action_string):
    self.action = action_string
    self.action_hash = hash(action_string)
    return

  def print_info(self):
    print(self.action, self.action_hash)
    return

  @classmethod
  def action_from(self, action_string):
    if action_string == "reassign":
      return Reassign()

In [309]:
class Reassign(Action):

  def __init__(self):
    super().__init__("reassign")
    return

  def run(self, state, chapter):
    chapter.reassign_leaders()
    new_state = chapter.produce_state()
    return new_state

In [310]:
class Promote_cascade(Action):

  def __init__(self):
    super().__init__("promote_cascade")
    return

  def run(self, state):
    new_state = state.state_copy()
    return new_state

In [311]:
class Promote_leaders:

  def __init__(self):
    super().__init__("promote_leaders")
    return

  def run(self, state):
    new_state = state.state_copy()
    return new_state

In [312]:
class Promote_junior_grades:

  def __init__(self):
    super().__init__("promote_junior_grades")
    return

  def run(self, state):
    new_state = state.state_copy()
    return new_state

In [313]:
class Hire:

  def __init__(self):
    super().__init__("hire")
    return

  def run(self, state):
    new_state = state.state_copy()
    return new_state

In [314]:
class Hire_junior_grades:

  def __init__(self):
    super().__init__("hire_junior_grades")
    return

  def run(self, state):
    new_state = state.state_copy()
    return new_state

# Reward

In [315]:
class Reward:

  environment_actions = [
      "reassign",
      "promote_cascade",
      "promote_leaders",
      "promote_junior_grades",
      "hire",
      "hire_junior_grades"
  ]

  environment_state_action_reward = {
      (True, True, False, True, False)    : {"reassign":1},
      (True, True, False, False, False)   : {"reassign":1},

      (True, True, True, True, False)     : {"reassign":1},
      (True, True, True, False, False)    : {"reassign":1},

      (True, False, False, True, False)   : {"promote_cascade": 1, "promote_junior_grades": 1, "hire_junior_grades": 1},
      (True, False, False, False, False)  : {"promote_cascade": 1, "promote_junior_grades": 1, "hire_junior_grades": 1},

      (True, False, True, True, False)    : {"promote_cascade" : 1, "promote_junior_grades" : 1},
      (True, False, True, False, False)   : {"promote_cascade" : 1, "promote_junior_grades" : 1},

      (False, True, False, True, False)   : {"success" : 100},
      (False, True, False, False, False)  : {"success" : 100},

      (False, True, True, True, False)    : {"success" : 100},
      (False, True, True, False, False)   : {"success" : 100},

      (False, False, False, True, False)  : {"success" : 100},
      (False, False, False, False, False) : {"success" : 100},

      (False, False, True, True, False)    : {"success" : 100},
      (False, False, True, False, False)   : {"success" : 100},

      (True, True, False, True, True)    : {"reassign":1},
      (True, True, False, False, True)   : {"reassign":1},

      (True, True, True, True, True)     : {"reassign":1},
      (True, True, True, False, True)    : {"reassign":1},

      (True, False, False, True, True)   : {"promote_cascade": 1, "promote_junior_grades": 1, "hire_junior_grades": 1},
      (True, False, False, False, True)  : {"promote_cascade": 1, "promote_junior_grades": 1, "hire_junior_grades": 1},

      (True, False, True, True, True)    : {"promote_cascade" : 1, "promote_junior_grades" : 1},
      (True, False, True, False, True)   : {"promote_cascade" : 1, "promote_junior_grades" : 1},

      (False, True, False, True, True)   : {"promote_leaders": 1},
      (False, True, False, False, True)  : {"promote_leaders": 1},

      (False, True, True, True, True)    : {"promote_leaders": 1},
      (False, True, True, False, True)   : {"promote_leaders": 1},

      (False, False, False, True, True)  : {"promote_leaders": 1},
      (False, False, False, False, True) : {"promote_leaders": 1},

      (False, False, True, True, True)    : {"promote_leaders": 1},
      (False, False, True, False, True)   : {"promote_leaders": 1},
        }

  def __init__(self):
    return  

  def reward_for_state_action(self, state, action):
    state_tuple = state.state_tuple()
    action_string = action.action

    reward_actions = self.environment_state_action_reward[state_tuple]

    if "success" in reward_actions:
      return reward_actions["success"]

    if action_string in reward_actions:
      return reward_actions[action_string]
    
    return -1


# State Tests

In [316]:
class State_Test(unittest.TestCase):

    def test_state_tuple(self):
      st = State()
      st_tuple = st.state_tuple()
      self.assertTrue(st_tuple == (False, False, False, False, False))

    def test_hash_exists(self):
      st = State()
      st_hash = st.state_hash()
      self.assertTrue(st_hash > 0)

    def test_hash_exists(self):
      st = State()
      st_cp = st.state_copy()
      st_hash = st.state_hash()
      st_cp_hash = st_cp.state_hash()
      self.assertTrue(st_hash == st_cp_hash)

# unittest.main(verbosity=2, exit=False)
suite = unittest.TestLoader().loadTestsFromTestCase(State_Test)
unittest.TextTestRunner().run(suite)


..
----------------------------------------------------------------------
Ran 2 tests in 0.003s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

# Reward Tests

In [317]:
class Reward_Tests(unittest.TestCase):

    def test_reward_action(self):
      rw = Reward()
      st = State(*(True, True, False, True, False))
      ac = Action("reassign")   
      rw_val = rw.reward_for_state_action(st, ac)
      self.assertTrue(rw_val == 1)

    def test_punish_action(self):
      rw = Reward()
      st = State(*(True, True, False, True, False))
      ac = Action("promote_cascade")   
      rw_val = rw.reward_for_state_action(st, ac)
      self.assertTrue(rw_val == -1)

    def test_success_state(self):
      rw = Reward()
      st = State(*(False, True, False, True, False))
      ac = Action("promote_cascade")   
      rw_val = rw.reward_for_state_action(st, ac)
      self.assertTrue(rw_val == 100)

# unittest.main(verbosity=2, exit=False)
suite = unittest.TestLoader().loadTestsFromTestCase(Reward_Tests)
unittest.TextTestRunner().run(suite)

...
----------------------------------------------------------------------
Ran 3 tests in 0.005s

OK


<unittest.runner.TextTestResult run=3 errors=0 failures=0>

# Chapter Tests

In [318]:

class Chapter_Tests(unittest.TestCase):

    def setUp(self):
      self.ch = Chapter(people_gradewise = {"3":3, "4.1":3, "4.2":4, "4.3":6, "4.4":5, "5.1":1, "5.2":2, "MS":100})

    def test_total_people(self):
      self.assertTrue(self.ch.total_people() == 124)

    def test_total_leaders_leading(self):
      self.assertTrue(self.ch.allocated_leads.total_leaders_leading() == 0)

    def test_total_leaders_that_can_be_reassigned(self):
      self.assertTrue(self.ch.total_leaders_that_can_be_reassigned() == 10)

    def test_dict_leaders_that_can_be_reassigned(self):
      self.assertTrue(self.ch.dict_leaders_that_can_be_reassigned() == {'3': 3, '4.1': 3, '4.2': 4})

suite = unittest.TestLoader().loadTestsFromTestCase(Chapter_Tests)
unittest.TextTestRunner().run(suite)

class Chapter_Tests_After_Reassign(unittest.TestCase):

    def setUp(self):
      self.ch = Chapter(people_gradewise = {"3":3, "4.1":3, "4.2":4, "4.3":6, "4.4":5, "5.1":1, "5.2":2, "MS":100})
      self.ch.reassign_leaders()

    def test_total_leaders_leading_after_reassign(self):
      self.assertTrue(self.ch.allocated_leads.total_leaders_leading() == 10)

    def test_dict_successor_per_lead(self):
      self.assertTrue(self.ch.successor.dict_successor_per_lead() == {'3': 0.0, '4.1': 0.0, '4.2': 6.0})

    def test_dict_successor_per_engineer(self):
      self.assertTrue(self.ch.successor.dict_successor_per_engineer() == {'4.3': 0.8333333333333334, '4.4': 0.2, '5.1': 2.0})

    def test_total_successors_for_lead_grades(self):
      self.assertTrue(self.ch.successor.total_successors_for_lead_grades() == 6.0)

    def test_total_successors_for_engineer_grades(self):
      self.assertTrue(self.ch.successor.total_successors_for_engineer_grades() == 3.033333333333333)

    def test_diff_successor_ratio_for_leader_engineer_grades(self):
      self.assertTrue(self.ch.successor.diff_successor_ratio_for_leader_engineer_grades() == 2.966666666666667)

    def test_total_people_remaining(self):
      self.assertTrue(self.ch.dr_per_lead.total_people_remaining() == 114)

    #state 
    def test_is_overall_dr_per_lead_gt_target(self):
      self.assertTrue(self.ch.dr_per_lead.dr_per_lead() == 11.4)
      self.assertTrue(self.ch.is_overall_dr_per_lead_gt_target() == True)

    def test_is_num_reassignable_leaders_gt_zero(self):
      self.assertTrue(self.ch.total_leaders_that_can_be_reassigned() == 0)
      self.assertTrue(self.ch.is_num_reassignable_leaders_gt_zero() == False)

    def test_is_num_immediate_promotable_to_leader_grades_gt_zero(self):
      self.assertTrue(self.ch.successor.total_successors_for_lead_grades() == 6)
      self.assertTrue(self.ch.successor.is_num_immediate_promotable_to_leader_grades_gt_zero() == True)

    def test_is_max_dr_per_lead_in_leader_grades_gt_target(self):
      self.assertTrue(self.ch.dr_per_lead_for_this_leader_grade_wrt_higher_grades.max_dr_per_lead_in_leader_grades() == 1)
      self.assertTrue(self.ch.dr_per_lead_for_this_leader_grade_wrt_higher_grades.dict_dr_per_lead_for_this_grade_wrt_higher_grade() == {'3': 0, '4.1': 1.0, '4.2': 0.6666666666666666})
      self.assertTrue(self.ch.dr_per_lead_for_this_leader_grade_wrt_higher_grades.is_max_dr_per_lead_in_leader_grades_gt_target() == False)

    def test_produce_state(self):
      st=ch.produce_state()
      self.assertTrue(st.state_tuple() == (True, False, True, True, False))

suite = unittest.TestLoader().loadTestsFromTestCase(Chapter_Tests_After_Reassign)
unittest.TextTestRunner().run(suite)



....
----------------------------------------------------------------------
Ran 4 tests in 0.007s

OK
............
----------------------------------------------------------------------
Ran 12 tests in 0.032s

OK


{'3': 3, '4.1': 3, '4.2': 4, '4.3': 6, '4.4': 5, '5.1': 1, '5.2': 2, 'MS': 100}


<unittest.runner.TextTestResult run=12 errors=0 failures=0>

In [319]:
class Reassign_Tests(unittest.TestCase):

    def setUp(self):
      self.ch = Chapter(people_gradewise = {"3":3, "4.1":3, "4.2":4, "4.3":6, "4.4":5, "5.1":1, "5.2":2, "MS":100})

    def test_before_reassign(self):
      st = self.ch.produce_state()
      self.assertTrue(st.state_tuple() == (True, True, True, True, False))

    def test_after_reassign(self):
      action = Reassign()
      st2 = action.run(st, self.ch)
      self.assertTrue(st2.state_tuple() == (True, False, True, True, False))


suite = unittest.TestLoader().loadTestsFromTestCase(Reassign_Tests)
unittest.TextTestRunner().run(suite)

..
----------------------------------------------------------------------
Ran 2 tests in 0.006s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>